# 搭建Minimind模型
首先要理清模型结构<br>
![structure](../images/LLM-structure.png)


# 搭建思路
采用自底向上的方式搭建model，理清所有的因素
- tokenizer
- embedding
- AttentionBlock
- ffn
- output


# tokenizer
在此之前已经训练好了tokenizer了，这里我们就开始利用之前训练好的tokenizer，来将原始信息转为input_ids的结构<br>
这里学习到的点：padding来统一输出input_ids的形状。left-padding和right-padding也是常见考察点<br>
注意！tokenizer初始化的时候就定好了padding的方向了，后续无法更改的。

In [ ]:
from numpy import pad
from transformers import AutoTokenizer
tokenizer_path="./"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path,padding_side="right") # padding_side可以是"left"或"right"，默认是"right"
#单条信息的情况,batch=1
messages = [[
        {"role": "system", "content": "你是一个优秀的聊天机器人，总是给我正确的回应！"},
        {"role": "user", "content": '你来自哪里？'},
        {"role": "assistant", "content": '我来自地球'}
    ],[
        {"role": "system", "content": "你是一个糟糕的捣乱机器人，总是给我错误的回应！"},
        {"role": "user", "content": '你来自哪里？'},
        {"role": "assistant", "content": '我来自火星'}
    ],
    ]
#多条信息

inputs_r = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True,return_tensors="pt",padding="max_length",
    truncation=True,
    max_length=100,
    padding_side="right"# 可以尝试"left"或"right"
    ) #直接使用会导致长度不一致的典型问题，因此需要padding到一致长度
print("inputs_r.shape=",inputs_r.shape)
print("inputs_r",inputs_r)
inputs_l = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True,return_tensors="pt",padding="max_length",
    truncation=True,
    max_length=100,
    padding_side="left"# 可以尝试"left"或"right"
    )
print("inputs_l.shape=",inputs_l.shape)
print("inputs_l",inputs_l)

#由于一开始tokenizer初始化的时候就定好了padding的方向了，所以后续无法更改的。上面也对比出来了

input_ids=inputs_r
print(input_ids.shape)
print(type(input_ids))
# 可知有很多tokenizer输出的tensor形状了



inputs_r.shape= torch.Size([2, 100])
inputs_r tensor([[   1,   87,   93,  307,   73,   81,  203,  397,  924, 5235, 3317, 2117,
          265, 2603, 1132, 2599,  703,  472,  997,    2,  203,    1,   89,   87,
         3709,  203,  397, 2722, 3016,  425,    2,  203,    1,   69,   87,   87,
           77,  307, 3924,   88,  203,  301, 2722, 1284,    2,  203,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [   1,   87,   93,  307,   73,   81,  203,  397,  924, 5990, 2391,  328,
          240,  101, 3789, 2117,  265, 2603, 1132, 1395,  264,  703,  472,  997,
            2,  203,    1,   89,   87, 3709,  203,  397, 2722, 3016,  425,    2,
          203,    1,   69,   

# embedding
tokenizer实现了word2vec，而下一步就是将原始的类似one-hot编码转化为向量化的压缩的input_ids

In [ ]:
#实现embedding
import torch
from torch import nn
# embedding
class Embedding(nn.Module):
    def __init__(self,vocab_size,embed_dim):
        super(Embedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
    def forward(self,input_ids):
        return self.embedding(input_ids)

# 测试embedding
vocab_size = tokenizer.vocab_size
embed_dim = 128  # 嵌入维度
embedding_layer = Embedding(vocab_size, embed_dim)
# input_ids是tokenizer输出的input_ids
res= embedding_layer(input_ids)
print("res.shape=",res.shape)
print("res",res)



res.shape= torch.Size([2, 100, 128])
res tensor([[[-0.3168,  0.9605, -0.6649,  ..., -0.4188, -0.7787, -0.3694],
         [ 0.2875,  0.8859,  0.8194,  ...,  1.5720,  0.2437, -1.4731],
         [ 2.0624,  2.5026, -0.3948,  ..., -0.8403,  0.6490,  1.8705],
         ...,
         [-0.1398, -1.4757,  2.4973,  ...,  0.9886,  1.8192,  0.4483],
         [-0.1398, -1.4757,  2.4973,  ...,  0.9886,  1.8192,  0.4483],
         [-0.1398, -1.4757,  2.4973,  ...,  0.9886,  1.8192,  0.4483]],

        [[-0.3168,  0.9605, -0.6649,  ..., -0.4188, -0.7787, -0.3694],
         [ 0.2875,  0.8859,  0.8194,  ...,  1.5720,  0.2437, -1.4731],
         [ 2.0624,  2.5026, -0.3948,  ..., -0.8403,  0.6490,  1.8705],
         ...,
         [-0.1398, -1.4757,  2.4973,  ...,  0.9886,  1.8192,  0.4483],
         [-0.1398, -1.4757,  2.4973,  ...,  0.9886,  1.8192,  0.4483],
         [-0.1398, -1.4757,  2.4973,  ...,  0.9886,  1.8192,  0.4483]]],
       grad_fn=<EmbeddingBackward0>)


# AttentionBlock
主要分为三个组件
- RMSNorm
- GQA
- FFN
embedding结束后，输入向量压缩为了[2,100,128]的tensor<br>
将这个tensor输入attentionblock，捕捉tensor内部的注意力关系<br>
这里采用的GQA机制，需要实现GQA，还需要实现RoPE编码，从而捕捉tensor内部的位置和时序关系
![img](../images/LLM-structure.png)

## RMSNorm
### 均方根层归一化 (Root Mean Square Layer Normalization, RMSNorm)

RMSNorm 是对 LayerNorm 的一个改进,  没有做 re-center 操作（移除了均值项）, 可以看作 LayerNorm 在均值为零时的特例, 使用平方根均值归一化降低噪声影响。

- **Layer Norm**

$$y = \frac{x-E(x)}{\sqrt{Var(x) + \epsilon}} * \gamma + \beta$$

假设输入张量形状为 (batch_size,  sequence_length,  embedding_dim), 层归一化对 embedding_dim 维度进行归一化操作, 其中,  $\epsilon$ 是一个超参数, 用于防止分母为零导致结果上溢,  $\gamma$,  $\beta$ 均为可学习参数。

- **RMS Norm**

$$a_i=\frac{a_i}{RMS(a) + \epsilon} * \gamma,  \quad where \quad RMS(a) = \sqrt{\frac{1}{n}\sum^n_{i=1}a^2_i}.$$

假设输入张量形状为 (batch_size,  sequence_length,  embedding_dim), RMS Norm 对 embedding_dim 维度进行归一化,其中,  其中,  $\epsilon$ 是一个超参数, 用于防止分母为零导致结果上溢, $\gamma$ 为可学习参数.

不难发现, 当均值为零时, Layer Norm 退化为 RMS Norm. 这是因为 RMS Norm 在 Layer Norm 的基础上舍弃了中心化操作, 仅用缩放进行归一化, 其不改变数据原本的分布, 有利于激活函数输出的稳定.

In [26]:
class RMSNorm(nn.Module):
    def __init__(self,embed_dim,eps=1e-6):
        super(RMSNorm,self).__init__()
        self.embed_dim=embed_dim
        self.eps=eps
        self.weight=nn.Parameter(torch.ones(embed_dim))
    def forward(self,x):
        return x*torch.rsqrt(x.pow(2).mean(-1,keepdim=True)+self.eps).type_as(x)*self.weight
# 测试RMSNorm
embed_dim = 128  # 嵌入维度
rmsnorm_layer = RMSNorm(embed_dim)
input_tensor = torch.randn(2, 100, embed_dim)  # 假设输入的tensor形状为[batch_size, seq_length, embed_dim]
output_tensor = rmsnorm_layer(input_tensor)
print("input_tensor",input_tensor)
print("output_tensor",output_tensor)

input_tensor tensor([[[-0.0319, -0.8763, -0.0228,  ...,  0.2425,  1.8407,  1.4291],
         [-0.9328,  0.3192, -0.7244,  ..., -0.1701,  0.1923, -0.9579],
         [-0.8071,  0.0079, -0.8318,  ...,  0.2133, -1.6108, -0.6445],
         ...,
         [-0.2219,  2.1274,  0.8018,  ..., -0.2066,  0.1366, -0.2633],
         [ 0.0508, -0.4104, -1.8380,  ...,  0.2081,  0.4585, -0.5310],
         [-0.2338,  1.0565, -0.0239,  ..., -0.3222,  1.2725,  0.9124]],

        [[ 0.5747, -0.2042, -0.8613,  ..., -0.4730, -1.0729,  0.4904],
         [ 1.9128,  0.8612,  1.1180,  ...,  0.6216,  0.4002,  0.2998],
         [-2.3053, -0.4935, -0.4944,  ..., -1.4535,  0.8217,  0.1968],
         ...,
         [-0.7073, -1.4368,  0.3273,  ..., -0.4203,  0.8883, -0.1376],
         [-0.1859,  1.2741, -0.1814,  ..., -0.3975, -0.6768, -0.5402],
         [ 1.8041, -0.5722, -1.4134,  ...,  1.3582, -0.2085, -1.2377]]])
output_tensor tensor([[[-0.0305, -0.8394, -0.0218,  ...,  0.2323,  1.7632,  1.3689],
         [-0.9186,

## RoPE
### Rotary Position Embedding, RoPE

旋转位置编码是一种能将相对位置信息集成到 self-attention 中, 进而提升 transformer 架构性能的位置编码方式, 和绝对位置编码相比, RoPE 具有很好的外推性, 是目前的主流位置编码方式.

外推性的解释, 通俗来说就是训练的时候限制了 512 的上下文长度，那么推理时如果面对超过该长度的文本，LLM 可能无法正确处理.

- **绝对位置编码**

绝对位置编码是早期 Transformer 架构采用的绝对位置编码方案，及那个每个位置映射为固定的向量表示.

$$f_{t:t\in\{q,k,v\}}(\boldsymbol{x}_i,i)=\boldsymbol{W}_{t:t\in\{q,k,v\}}(\boldsymbol{x}_i+\boldsymbol{p}_i)$$

其中编码向量 $p_i$ 的计算使用如下公式：

$$\boldsymbol{p}_{i,2t}=\sin\left(k/1000^{2t/d}\right), \boldsymbol{p}_{i,2t+1}=\cos\left(k/1000^{2t/d}\right)$$

正如其名，绝对位置编码只考虑了输入序列中的绝对位置关系，对于 token 之间的相对信息则没有纳入考虑.

- **旋转位置编码**

假定 query 和 key 的内积操作可以被函数 g 表示，该函数 g 的输入是词嵌入向量 $x_m, x_n$ 和它们之间的相对位置 $m-n$:

$$<f_q(x_m ,m), f_k(x_n, n)>=g(x_m, x_n, m, n)$$

旋转位置编码就是找到一个使上式成立的位置编码方式. 

出于认识的目的，我们省略复杂的数学推导，直接看 RoPE 的的结论：

存在这样一个正交矩阵：

$$\boldsymbol{R}_{\Theta,m}^d=\underbrace{\begin{pmatrix}\cos m\theta_0&-\sin m\theta_0&0&0&\cdots&0&0\\\sin m\theta_0&\cos m\theta_0&0&0&\cdots&0&0\\0&0&\cos m\theta_1&-\sin m\theta_1&\cdots&0&0\\0&0&\sin m\theta_1&\cos m\theta_1&\cdots&0&0\\\vdots&\vdots&\vdots&\vdots&\ddots&\vdots&\vdots\\0&0&0&0&\cdots&\cos m\theta_{d/2-1}&-\sin m\theta_{d/2-1}&-\sin m\theta_{d/2-1}\end{pmatrix}}_{\boldsymbol{W}_m}$$

其中，$\Theta=\left\{\theta_i=10000^{-2(i-1)/d},i\in[1,2,\ldots,d/2]\right\}$

我们可以将 query 和 key 的内积操作转换为与原始向量 $x$ 相关的以下等价形式：

$$
\boldsymbol{q}_m^\mathbf{T}\boldsymbol{k}_n=\left(\boldsymbol{R}_{\Theta,m}^d\boldsymbol{W}_q\boldsymbol{x}_m\right)^\mathbf{T}\left(\boldsymbol{R}_{\Theta,n}^d\boldsymbol{W}_k\boldsymbol{x}_n\right)=\boldsymbol{x}_m^\mathbf{T}\boldsymbol{W}_q\boldsymbol{R}_{\Theta,n-m}^d\boldsymbol{W}_k\boldsymbol{x}_n
$$

其中， $\boldsymbol{R}_{\Theta,n-m}^d=\left(\boldsymbol{R}_{\Theta,m}^d\right)^\mathbf{T}\boldsymbol{R}_{\Theta,n}^d$.

由于 $\boldsymbol{R}_{\Theta,m}^d$ 的稀疏性，直接使用矩阵乘法会浪费算力，因此代码中采用下述方式实现：

$$\boldsymbol{R}_{\Theta,m}^{d}\boldsymbol{x}=\begin{pmatrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\\vdots\\x_{d-2}\\x_{d-1}\end{pmatrix}\otimes\begin{pmatrix}\cos m\theta_{0}\\\cos m\theta_{0}\\\cos m\theta_{1}\\\cos m\theta_{1}\\\vdots\\\cos m\theta_{d/2-1}\\\cos m\theta_{d/2-1}\end{pmatrix}+\begin{pmatrix}-x_{1}\\x_{0}\\-x_{3}\\x_{2}\\\vdots\\-x_{d-1}\\x_{d-2}\end{pmatrix}\otimes\begin{pmatrix}\sin m\theta_{0}\\\sin m\theta_{0}\\\sin m\theta_{1}\\\sin m\theta_{1}\\\vdots\\\sin m\theta_{d/2-1}\\\sin m\theta_{d/2-1}\end{pmatrix}
$$

简而言之，RoPE就是用绝对编码的形式，表示出相对编码的关系，这样同时具有了绝对编码的简洁和相对编码的位置信息泛化性<br>
此处的ROPE的实现主要参考的是LLama的RoPE实现
[LLAMA实现](https://blog.csdn.net/m0_55846238/article/details/145728695)<br>
对旋转编码理解困难，可以参考[无痛理解RoPE](https://zhuanlan.zhihu.com/p/8306958113)


大概归纳一下，旋转编码主要两步，首先是制作 $m\Theta$ 的旋转角度的表，之后再应用这个表，用于编码qk



### 首先制作$m\Theta$

In [34]:
def precompute_pos_cis(dim,seqlen,theta=1e5):
    #这里//2是因为要把dim分成两半，前半部分用于cos，后半部分用于sin，所以只会用到一个theta
    freqs=1.0/(theta**(torch.arange(0,dim,2)[:dim//2].float()/dim))
    print("freqs.shape=",freqs.shape)
    
    m=torch.arange(seqlen,device=freqs.device)
    print("m.shape=",m.shape)

    freqs=torch.outer(m,freqs).float()
    print("freqs.shape=",freqs.shape)

    pos_cis=torch.polar(torch.ones_like(freqs),freqs)
    print("pos_cis.shape=",pos_cis.shape)
    return pos_cis

# 测试precompute_pos_cis
dim = 128  # 嵌入维度
seqlen = 100  # 序列长度
pos_cis = precompute_pos_cis(dim, seqlen)
print(type(pos_cis))

freqs.shape= torch.Size([64])
m.shape= torch.Size([100])
freqs.shape= torch.Size([100, 64])
pos_cis.shape= torch.Size([100, 64])
<class 'torch.Tensor'>


### 然后将$m\Theta$应用到旋转编码计算中去

In [35]:
def apply_rotary_emb(xq,xk,pos_cis):
    xq_=torch.view_as_complex(xq.float().reshape(*xq.shape[:-1],-1,2))
    xk_=torch.view_as_complex(xk.float().reshape(*xk.shape[:-1],-1,2))
    #由于view_as_complex,最后一维合并了，就变成了dim//2的形状
    print("xq_.shape=",xq_.shape)
    def unite_shape(pos_cis,x):
        #将pos_cis对齐x的形状
        ndim = x.ndim
        assert 0 <= 1 < ndim
        #x形状一般为(batch_size, seq_len, n_heads, head_dim//2)
        #这里确保freqs_cis与x的seq_len, head_dim//2维度一致, RoPE是对每个头分别进行的
        assert pos_cis.shape == (x.shape[1],  x.shape[-1]),f"pos_cis.shape:({pos_cis.shape}),(x.shape[1],  x.shape[-1])={(x.shape[1],  x.shape[-1])}"
        shape = [d if i == 1 or i == ndim - 1 else 1 for i,  d in enumerate(x.shape)]
        return pos_cis.view(*shape)
    pos_cis = unite_shape(pos_cis, xq_)
    #将pos_cis应用到xq_和xk_上(和输入对齐)
    print("pos_cis shape:", pos_cis.shape)
    print("xq_ shape:", xq_.shape)
    print("xk_ shape:", xk_.shape)
    xq_out=torch.view_as_real(xq_ * pos_cis).flatten(3)
    xk_out=torch.view_as_real(xk_ * pos_cis).flatten(3)
    return xq_out, xk_out
#测试一下apply_rotary_emb函数
xq = torch.randn(2, 3, 2,8)  # (bs, seqlen, dim)
xk = torch.randn(2, 3, 2,8)  # (bs, seqlen, dim)
pos_cis = precompute_pos_cis(dim=8, seqlen=3, theta=1e5)
print(type(pos_cis))
xq_out, xk_out = apply_rotary_emb(xq, xk, pos_cis)
print("xq shape:", xq.shape)  # 应该是 (bs, seqlen, dim)
print("xk shape:", xk.shape)  # 应该是 (bs, seqlen, dim)

print("xq_out_shape:", xq_out.shape)  # 应该是 (bs, seqlen, dim
print("xk_out_shape:", xk_out.shape)  # 应该是 (bs, seqlen, dim
# print("xq_out:", xq_out)
# print("xk_out:", xk_out)
print("pos_cis:", pos_cis)
print("pos_cis shape:", pos_cis.shape)



freqs.shape= torch.Size([4])
m.shape= torch.Size([3])
freqs.shape= torch.Size([3, 4])
pos_cis.shape= torch.Size([3, 4])
<class 'torch.Tensor'>
xq_.shape= torch.Size([2, 3, 2, 4])
pos_cis shape: torch.Size([1, 3, 1, 4])
xq_ shape: torch.Size([2, 3, 2, 4])
xk_ shape: torch.Size([2, 3, 2, 4])
xq shape: torch.Size([2, 3, 2, 8])
xk shape: torch.Size([2, 3, 2, 8])
xq_out_shape: torch.Size([2, 3, 2, 8])
xk_out_shape: torch.Size([2, 3, 2, 8])
pos_cis: tensor([[ 1.0000+0.0000e+00j,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
          1.0000+0.0000e+00j],
        [ 0.5403+8.4147e-01j,  0.9984+5.6204e-02j,  1.0000+3.1623e-03j,
          1.0000+1.7783e-04j],
        [-0.4161+9.0930e-01j,  0.9937+1.1223e-01j,  1.0000+6.3245e-03j,
          1.0000+3.5566e-04j]])
pos_cis shape: torch.Size([3, 4])
